In [1]:
import pandas as pd
import re
import os

In [2]:
df = pd.read_excel(r"MO14-Round-1-Dealing-With-Data-Workbook.xlsx",sheet_name = 'Usage',header=None)
df

/usr/local/lib/python3.10/dist-packages/openpyxl/worksheet/_reader.py:329: UserWarning: Unknown extension is not supported and will be removed
  warn(msg)


,0
0,3 PM Mon 24th-Mar-2014___0.384 kwh
1,5AM 15-Aug-2014___1.201 kwh
2,__8PM Thu 20-Mar-2014____1.523 kwh
3,6PM 23rd-Apr-2014___0.424 kwh
4,_1AM Friday 19th-Dec-2014___0.209 kwh
...,...
8755,_1AM Fri 07th-Nov-2014_0.084 kwh
8756,_6AM 20-May-2014__1.027 kwh
8757,__2 AM Tuesday 8th-Apr-2014___0.052 kwh
8758,9 PM 27th-Jan-2014_1.428 kwh


In [3]:
df.columns = ['Usage Data']
df

,Usage Data
0,3 PM Mon 24th-Mar-2014___0.384 kwh
1,5AM 15-Aug-2014___1.201 kwh
2,__8PM Thu 20-Mar-2014____1.523 kwh
3,6PM 23rd-Apr-2014___0.424 kwh
4,_1AM Friday 19th-Dec-2014___0.209 kwh
...,...
8755,_1AM Fri 07th-Nov-2014_0.084 kwh
8756,_6AM 20-May-2014__1.027 kwh
8757,__2 AM Tuesday 8th-Apr-2014___0.052 kwh
8758,9 PM 27th-Jan-2014_1.428 kwh


In [4]:
df['Usage Data'] = df['Usage Data'].str.replace('_','   ')
df

,Usage Data
0,3 PM Mon 24th-Mar-2014 0.384 kwh
1,5AM 15-Aug-2014 1.201 kwh
2,8PM Thu 20-Mar-2014 1.523 kwh
3,6PM 23rd-Apr-2014 0.424 kwh
4,1AM Friday 19th-Dec-2014 0.209 kwh
...,...
8755,1AM Fri 07th-Nov-2014 0.084 kwh
8756,6AM 20-May-2014 1.027 kwh
8757,2 AM Tuesday 8th-Apr-2014 0.05...
8758,9 PM 27th-Jan-2014 1.428 kwh


In [5]:
df['Energy value'] = df['Usage Data'].str.extract(r'(\d+\.\d+)').astype(float)
df['Unit'] = df['Usage Data'].str.extract(r'(kwh)')
df['Energy Consumption'] = df['Energy value'].astype(str) + ' ' + df['Unit']
df = df.drop(columns=['Energy value','Unit'])
df

,Usage Data,Energy Consumption
0,3 PM Mon 24th-Mar-2014 0.384 kwh,0.384 kwh
1,5AM 15-Aug-2014 1.201 kwh,1.201 kwh
2,8PM Thu 20-Mar-2014 1.523 kwh,1.523 kwh
3,6PM 23rd-Apr-2014 0.424 kwh,0.424 kwh
4,1AM Friday 19th-Dec-2014 0.209 kwh,0.209 kwh
...,...,...
8755,1AM Fri 07th-Nov-2014 0.084 kwh,0.084 kwh
8756,6AM 20-May-2014 1.027 kwh,1.027 kwh
8757,2 AM Tuesday 8th-Apr-2014 0.05...,0.052 kwh
8758,9 PM 27th-Jan-2014 1.428 kwh,1.428 kwh


In [6]:
df['Usage Data'] = df['Usage Data'].str.replace('kwh','').str.strip()
df['Usage Data'] = df['Usage Data'].str.replace(r'\d+\.\d+', '', regex=True).str.strip()
df

,Usage Data,Energy Consumption
0,3 PM Mon 24th-Mar-2014,0.384 kwh
1,5AM 15-Aug-2014,1.201 kwh
2,8PM Thu 20-Mar-2014,1.523 kwh
3,6PM 23rd-Apr-2014,0.424 kwh
4,1AM Friday 19th-Dec-2014,0.209 kwh
...,...,...
8755,1AM Fri 07th-Nov-2014,0.084 kwh
8756,6AM 20-May-2014,1.027 kwh
8757,2 AM Tuesday 8th-Apr-2014,0.052 kwh
8758,9 PM 27th-Jan-2014,1.428 kwh


In [7]:
def extract_date(text):
    date_patterns = [
        r'(\d{1,2})(st|nd|rd|th)?\s*-\s*(jan|feb|mar|apr|may|jun|jul|aug|sep|oct|nov|dec)\s*-\s*2014',
        r'(jan|feb|mar|apr|may|jun|jul|aug|sep|oct|nov|dec)\s*(\d{1,2})(st|nd|rd|th)?\s*,\s*2014'
    ]

    for pattern in date_patterns:
        match = re.search(pattern, text, re.IGNORECASE)
        if match:
            day = match.group(1).zfill(2)
            month = match.group(3).capitalize()
            year = "2014"
            date_str = f"{day}-{month}-{year}"
            return pd.to_datetime(date_str, format='%d-%b-%Y')

    return None


df['Date'] = df['Usage Data'].apply(extract_date)
df

,Usage Data,Energy Consumption,Date
0,3 PM Mon 24th-Mar-2014,0.384 kwh,2014-03-24
1,5AM 15-Aug-2014,1.201 kwh,2014-08-15
2,8PM Thu 20-Mar-2014,1.523 kwh,2014-03-20
3,6PM 23rd-Apr-2014,0.424 kwh,2014-04-23
4,1AM Friday 19th-Dec-2014,0.209 kwh,2014-12-19
...,...,...,...
8755,1AM Fri 07th-Nov-2014,0.084 kwh,2014-11-07
8756,6AM 20-May-2014,1.027 kwh,2014-05-20
8757,2 AM Tuesday 8th-Apr-2014,0.052 kwh,2014-04-08
8758,9 PM 27th-Jan-2014,1.428 kwh,2014-01-27


In [8]:
df['Usage Data'] = df['Usage Data'].str.replace('-','   ')
df['Usage Data'] = df['Usage Data'].str.replace('2014','   ').str.strip()
months = ['Jan','Feb','Mar','Apr','May','Jun','Jul','Aug','Sep','Oct','Nov','Dec']
for month in months:
    df['Usage Data'] = df['Usage Data'].str.replace(month,' ').str.strip()
df['Usage Data'] = df['Usage Data'].str.replace(r'\b\d{1,2}(st|nd|rd|th)?\b(?!\s?(AM|PM|am|pm))', '', regex=True).str.strip()
df

,Usage Data,Energy Consumption,Date
0,3 PM Mon,0.384 kwh,2014-03-24
1,5AM,1.201 kwh,2014-08-15
2,8PM Thu,1.523 kwh,2014-03-20
3,6PM,0.424 kwh,2014-04-23
4,1AM Friday,0.209 kwh,2014-12-19
...,...,...,...
8755,1AM Fri,0.084 kwh,2014-11-07
8756,6AM,1.027 kwh,2014-05-20
8757,2 AM Tuesday,0.052 kwh,2014-04-08
8758,9 PM,1.428 kwh,2014-01-27


In [9]:
df['Time'] = df['Usage Data'].str.extract(r'(\b\d{1,2}\s?[APM]{2}\b)', expand=False)
df['Usage Data'] = df['Usage Data'].str.replace(r'\b\d{1,2}\s?[APM]{2}\b', '', regex=True).str.strip()
df

,Usage Data,Energy Consumption,Date,Time
0,Mon,0.384 kwh,2014-03-24,3 PM
1,,1.201 kwh,2014-08-15,5AM
2,Thu,1.523 kwh,2014-03-20,8PM
3,,0.424 kwh,2014-04-23,6PM
4,Friday,0.209 kwh,2014-12-19,1AM
...,...,...,...,...
8755,Fri,0.084 kwh,2014-11-07,1AM
8756,,1.027 kwh,2014-05-20,6AM
8757,Tuesday,0.052 kwh,2014-04-08,2 AM
8758,,1.428 kwh,2014-01-27,9 PM


In [10]:
df['Day'] = df['Date'].dt.day_name()

df

,Usage Data,Energy Consumption,Date,Time,Day
0,Mon,0.384 kwh,2014-03-24,3 PM,Monday
1,,1.201 kwh,2014-08-15,5AM,Friday
2,Thu,1.523 kwh,2014-03-20,8PM,Thursday
3,,0.424 kwh,2014-04-23,6PM,Wednesday
4,Friday,0.209 kwh,2014-12-19,1AM,Friday
...,...,...,...,...,...
8755,Fri,0.084 kwh,2014-11-07,1AM,Friday
8756,,1.027 kwh,2014-05-20,6AM,Tuesday
8757,Tuesday,0.052 kwh,2014-04-08,2 AM,Tuesday
8758,,1.428 kwh,2014-01-27,9 PM,Monday


In [11]:
df['Time'] = df['Time'].str.replace(' ', '')
def transform_time(time_str):
    match = re.match(r'(\d{1,2})(AM|PM)', time_str)
    if match:
        hour = int(match.group(1))
        period = match.group(2)
        return f"{hour}:00 {period} to {hour}:59 {period}"
    return None

df['Time'] = df['Time'].apply(transform_time)
df

,Usage Data,Energy Consumption,Date,Time,Day
0,Mon,0.384 kwh,2014-03-24,3:00 PM to 3:59 PM,Monday
1,,1.201 kwh,2014-08-15,5:00 AM to 5:59 AM,Friday
2,Thu,1.523 kwh,2014-03-20,8:00 PM to 8:59 PM,Thursday
3,,0.424 kwh,2014-04-23,6:00 PM to 6:59 PM,Wednesday
4,Friday,0.209 kwh,2014-12-19,1:00 AM to 1:59 AM,Friday
...,...,...,...,...,...
8755,Fri,0.084 kwh,2014-11-07,1:00 AM to 1:59 AM,Friday
8756,,1.027 kwh,2014-05-20,6:00 AM to 6:59 AM,Tuesday
8757,Tuesday,0.052 kwh,2014-04-08,2:00 AM to 2:59 AM,Tuesday
8758,,1.428 kwh,2014-01-27,9:00 PM to 9:59 PM,Monday


In [12]:
df = df.sort_values(by=['Date','Time']).reset_index(drop=True)
df['Date'] = df['Date'].dt.date
df

,Usage Data,Energy Consumption,Date,Time,Day
0,Wednesday,0.952 kwh,2014-01-01,10:00 AM to 10:59 AM,Wednesday
1,,1.2 kwh,2014-01-01,10:00 PM to 10:59 PM,Wednesday
2,,0.83 kwh,2014-01-01,11:00 AM to 11:59 AM,Wednesday
3,,1.13 kwh,2014-01-01,11:00 PM to 11:59 PM,Wednesday
4,,0.684 kwh,2014-01-01,12:00 AM to 12:59 AM,Wednesday
...,...,...,...,...,...
8755,,1.94 kwh,2014-12-31,7:00 PM to 7:59 PM,Wednesday
8756,,0.361 kwh,2014-12-31,8:00 AM to 8:59 AM,Wednesday
8757,Wednesday,1.901 kwh,2014-12-31,8:00 PM to 8:59 PM,Wednesday
8758,,0.32 kwh,2014-12-31,9:00 AM to 9:59 AM,Wednesday


In [14]:
cleanised_data = df[['Date', 'Time', 'Energy Consumption','Day']]
cleanised_data

,Date,Time,Energy Consumption,Day
0,2014-01-01,10:00 AM to 10:59 AM,0.952 kwh,Wednesday
1,2014-01-01,10:00 PM to 10:59 PM,1.2 kwh,Wednesday
2,2014-01-01,11:00 AM to 11:59 AM,0.83 kwh,Wednesday
3,2014-01-01,11:00 PM to 11:59 PM,1.13 kwh,Wednesday
4,2014-01-01,12:00 AM to 12:59 AM,0.684 kwh,Wednesday
...,...,...,...,...
8755,2014-12-31,7:00 PM to 7:59 PM,1.94 kwh,Wednesday
8756,2014-12-31,8:00 AM to 8:59 AM,0.361 kwh,Wednesday
8757,2014-12-31,8:00 PM to 8:59 PM,1.901 kwh,Wednesday
8758,2014-12-31,9:00 AM to 9:59 AM,0.32 kwh,Wednesday
